<h1>光頭傑夫製作</h1><img src="https://bl3302files.storage.live.com/y4mccLq6gAm5ULg2cdCQ-Bl3y0Lw7mmtkvcdqInwwg3MAPeDpabT_YVDs7vVN8tvdaw4meL8i33B42YbUECTNMGkmykUQOC9uJxAC6jE_XkaP7K4BTP1xFvyONy_h8LOGV_QW_2CqIwThrYu1JxnujMB7IUR3y7NLJXaXRZMQekQL80-WZuuGw7EIsdneNGUnE0?width=100&height=91&cropmode=none" style='width:100px;margin:0'/>

### <span style="color:red ">設定目標股票
----
### [查詢股票代碼](https://tw.stock.yahoo.com/h/kimosel.php)

In [1]:
#########################################
#請填下面資料
from datetime import date,timedelta
目標股票代碼='2330.tw'
分析開始日期='2021-1-1'
分析結束日期=str(date.today() + timedelta(1))

#設立投資本金
#你可以根據你想要模擬的投資標的股價作適當調整
cash=10000000

##########################################

### <span style="color:red ">Colab安裝套件

In [ ]:
! pip install matplotlib==3.2.2
! pip install backtrader
! pip install yfinance

In [ ]:
import matplotlib
matplotlib.__version__

### <span style="color:red ">以下是我們後續輸出分析結果需要的方程式:
---    
    1. 每次你要輸出策略的績效結果時，下面這格你都要跑

In [2]:
def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    total_closed = ''
    total_won =''
    total_lost =''
    win_streak = ''
    lose_streak = ''
    pnl_net =''
    strike_rate =''
    
    if len(analyzer.total)>2:
        total_closed = analyzer.total.closed
        total_won = analyzer.won.total
        total_lost = analyzer.lost.total
        win_streak = analyzer.streak.won.longest
        lose_streak = analyzer.streak.lost.longest
        pnl_net = round(analyzer.pnl.net.total,2)
    
    if total_closed:
        strike_rate=round((total_won / total_closed) * 100,2)
    else:
        strike_rate='沒有完整交易'
    print("策略運作分析結果:")
    print(f'獲利累積次數 : {str(total_won)}')
    print(f'損失累積次數 : {str(total_lost)}')
    print(f'獲利損失次數比 : {str(strike_rate)}')
    print(f'最高連續獲利交易次數 : {str(win_streak)}')
    print(f'最高連續損失交易次數 : {str(lose_streak)}')
    print(f'淨利結算 : {str(pnl_net)}')
    

## <span style="color:red ">  <策略1-with Commision>用DMI做個回測策略吧!

### <span style="color:red "><策略1.1- with Commision>導入需求模組

In [3]:
from datetime import datetime
import pandas as pd
import yfinance as yf
import backtrader as bt

### <span style="color:red "><策略1.2- with Commision>建立交易策略

In [4]:
class DMIStrategy(bt.Strategy):
   
    # Standard DMI Parameters
    params = dict(
          ma_period=14, 
          adxLimited=25
    )

    def __init__(self):
        self.DMI = bt.indicators.DMI(period=self.p.ma_period)
                                       
        self.order = None
    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return

        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'買單執行 --- 買價: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手續費: {order.executed.comm:.2f}')
                self.price = order.executed.price
                self.comm = order.executed.comm
            else:
                self.log(f'賣單執行 --- 賣價: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 手續費: {order.executed.comm:.2f}')

        # report failed order
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失敗')

        # set no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'運作結果 --- 獲利狀況: {trade.pnl:.2f}, 淨獲利: {trade.pnlcomm:.2f}')

    def next(self):
        # do nothing if an order is pending
        if self.order:
            return
        
        # check if there is already a position
        if not self.position:
            if self.DMI.adx[0] > self.p.adxLimited and self.DMI.plusDI[0]>self.DMI.minusDI[0]: 
                self.log(f'開出買單 --- 當時市場價格: {self.data_close[0]:.2f}')
                self.order = self.buy(size=1000)

        else:
            # sell condition
            if self.DMI.adx[0] > self.p.adxLimited and self.DMI.plusDI[0]<self.DMI.minusDI[0]:
                self.log(f'開出賣單 --- 當時市場價格: {self.data_close[0]:.2f}')
                self.order = self.sell(size=1000)

### <span style="color:red "><策略1.3- with Commision>抓取目標股票的歷史資料

In [5]:
data= yf.download(  目標股票代碼,              #股票代碼
                    start=分析開始日期,      #資料開始日期
                    end=分析結束日期,         #資料結束日期
                    progress=False
                 )
data=bt.feeds.PandasData(dataname=pd.DataFrame({'open': data['Open'], 
                   'high': data['High'],
                   'low': data['Low'],
                   'close': data['Close'],
                   'volume': data['Volume'],
                  }))


### <span style="color:red "><策略1.4- with Commision>建立台股交易費用計算模式

In [6]:
class TWStockCommisionScheme(bt.CommInfoBase):
    params = (
        
         ('stocklike', True),
         ('commtype', bt.CommInfoBase.COMM_PERC),
         ('percabs', True),
         ('discount', 0.3),
         ('tax',0.003),
         ('lowestfee',20)
        )

    def _getcommission(self, size, price, pseudoexec):
          if size>0:
            return max(self.p.lowestfee,size * price * self.p.commission * self.p.discount)
          else:
            return (max(self.p.lowestfee,-(size * price * self.p.commission * self.p.discount))-size * price *self.p.tax)


### <span style="color:red "><策略1.5- with Commision>建立回測分析「大腦」

In [7]:
# create a Cerebro entity
cerebro = bt.Cerebro()

# set up the backtest
cerebro.adddata(data)
cerebro.broker.setcash(cash)
cerebro.addstrategy(DMIStrategy)

#commission手續費 0.1425%
#discount 你可以拿到的手續費折數
#tax=0.003 證券交易稅率
comminfo = TWStockCommisionScheme(
    commission=0.001425,  
    discount=0.5,
    tax=0.003,
    lowestfee=20
)

cerebro.broker.addcommissioninfo(comminfo)


### <span style="color:red "><策略1.6- with Commision>讓Analyzers來幫我診斷策略細節...-開始回測

In [8]:
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")

strategies = cerebro.run()
firstStrat = strategies[0]

# print the analyzers
printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())
print(f'持有資產總值: {cerebro.broker.getvalue():.2f}')

2021-02-19, 開出買單 --- 當時市場價格: 652.00
2021-02-22, 買單執行 --- 買價: 660.00, 成本: 660000.00, 手續費: 470.25
2021-03-05, 開出賣單 --- 當時市場價格: 601.00
2021-03-08, 賣單執行 --- 賣價: 608.00, 成本: 660000.00, 手續費: 2257.20
2021-03-08, 運作結果 --- 獲利狀況: -52000.00, 淨獲利: -54727.45
2022-01-05, 開出買單 --- 當時市場價格: 650.00
2022-01-06, 買單執行 --- 買價: 638.00, 成本: 638000.00, 手續費: 454.57
策略運作分析結果:
獲利累積次數 : 0
損失累積次數 : 1
獲利損失次數比 : 0.0
最高連續獲利交易次數 : 0
最高連續損失交易次數 : 1
淨利結算 : -54727.45
持有資產總值: 9943817.98


### <span style="color:red "><策略1.7- with Commision>畫張圖勝過千言萬語!-圖形化表達

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.dpi']=80
cerebro.plot(iplot=False)

[[<Figure size 1368x720 with 5 Axes>]]

## <span style="color:red "><策略2- with Commision>我們想要來個參數特調DMI看看效果怎麼樣?
--------

### <span style="color:red "><策略2.2- with Commision>建立交易策略

In [10]:
class DMIStrategy(bt.Strategy):
   
    # Standard DMI Parameters
    params = dict(
          ma_period=14, 
          adxLimited=25
    )

    def __init__(self):
        self.DMI = bt.indicators.DMI(period=self.p.ma_period)
                                       
        self.order = None
    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return

        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.price = order.executed.price
                self.comm = order.executed.comm
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return


    def next(self):
        # do nothing if an order is pending
        if self.order:
            return
        
        # check if there is already a position
        if not self.position:
            if self.DMI.adx[0] > self.p.adxLimited and self.DMI.plusDI[0]>self.DMI.minusDI[0]: 
                self.order = self.buy(size=1000)

        else:
            # sell condition
            if self.DMI.adx[0] > self.p.adxLimited and self.DMI.plusDI[0]<self.DMI.minusDI[0]:
                self.order = self.sell(size=1000)
    def stop(self):
        self.log(f'(DMI週期設定 = 週期:{self.p.ma_period:2d} ADX門檻設定 = {self.p.adxLimited:2d}) --- 淨利數值: {self.broker.getvalue()-cash:.2f}')
        self.params.totalValue=self.broker.getvalue()

### <span style="color:red "><策略2.4- with Commision>建立台股交易費用計算模式

In [11]:
class TWStockCommisionScheme(bt.CommInfoBase):
    params = (
        
         ('stocklike', True),
         ('commtype', bt.CommInfoBase.COMM_PERC),
         ('percabs', True),
         ('discount', 0.3),
         ('tax',0.003),
         ('lowestfee',20)
        )

    def _getcommission(self, size, price, pseudoexec):
          if size>0:
            return max(self.p.lowestfee,size * price * self.p.commission * self.p.discount)
          else:
            return (max(self.p.lowestfee,-(size * price * self.p.commission * self.p.discount))-size * price *self.p.tax)

### <span style="color:red "><策略2.5- with Commision>建立回測分析「大腦」,讓Analyzers來幫我診斷策略細節...-開始回測

In [14]:
# create a Cerebro entity
cerebro = bt.Cerebro()
cerebro.optstrategy(DMIStrategy,ma_period=range(10,20),adxLimited=[25,27,30,35])
cerebro.adddata(data)
cerebro.broker.setcash(cash)

#commission手續費 0.1425%
#discount 你可以拿到的手續費折數
#tax=0.003 證券交易稅率
comminfo = TWStockCommisionScheme(
    commission=0.001425,  
    discount=0.5,
    tax=0.003,
    lowestfee=20
)


cerebro.broker.addcommissioninfo(comminfo)

cerebro.addanalyzer(bt.analyzers.TradeAnalyzer,_name="ta")

opt_runs = cerebro.run(maxcpus=1)


for run in opt_runs:
    for strategy in run:
        print(f'參數組合:  週期={strategy.params.ma_period:2d}天,ADX門檻={strategy.params.adxLimited:2d}')
        try:
            printTradeAnalysis(strategy.analyzers.ta.get_analysis())
        except KeyError :
            print('沒有交易發生')
        print(f'持有資產總值: {strategy.params.totalValue:.2f}')
        print('--------------------------------------------------------')

2022-02-14, (DMI週期設定 = 週期:10 ADX門檻設定 = 25) --- 淨利數值: 17459.46
2022-02-14, (DMI週期設定 = 週期:10 ADX門檻設定 = 27) --- 淨利數值: -48188.72
2022-02-14, (DMI週期設定 = 週期:10 ADX門檻設定 = 30) --- 淨利數值: 18988.45
2022-02-14, (DMI週期設定 = 週期:10 ADX門檻設定 = 35) --- 淨利數值: -20329.54
2022-02-14, (DMI週期設定 = 週期:11 ADX門檻設定 = 25) --- 淨利數值: -69440.20
2022-02-14, (DMI週期設定 = 週期:11 ADX門檻設定 = 27) --- 淨利數值: -17211.76
2022-02-14, (DMI週期設定 = 週期:11 ADX門檻設定 = 30) --- 淨利數值: -5203.21
2022-02-14, (DMI週期設定 = 週期:11 ADX門檻設定 = 35) --- 淨利數值: -21214.61
2022-02-14, (DMI週期設定 = 週期:12 ADX門檻設定 = 25) --- 淨利數值: -24264.02
2022-02-14, (DMI週期設定 = 週期:12 ADX門檻設定 = 27) --- 淨利數值: -7898.04
2022-02-14, (DMI週期設定 = 週期:12 ADX門檻設定 = 30) --- 淨利數值: -23909.44
2022-02-14, (DMI週期設定 = 週期:12 ADX門檻設定 = 35) --- 淨利數值: -11207.49
2022-02-14, (DMI週期設定 = 週期:13 ADX門檻設定 = 25) --- 淨利數值: -43177.19
2022-02-14, (DMI週期設定 = 週期:13 ADX門檻設定 = 27) --- 淨利數值: -59188.59
2022-02-14, (DMI週期設定 = 週期:13 ADX門檻設定 = 30) --- 淨利數值: -64192.15
2022-02-14, (DMI週期設定 = 週期:13 ADX門檻設定 = 35) --- 淨利數值: -27473